In [ ]:
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
convini_sales=pd.read_excel("/Users/coj37/Documents/portfolio/time_series_analysis/convini_sales.xlsx")
index=pd.date_range("2008-01-31","2020-07-31",freq="M")
convini_sales.index=index
del convini_sales["Month"]
#convini_sales.head()

In [ ]:
#パラメータ決定関数の定義
#時系列データ：dataとパラメータs(周期)：s(今回は12と設定)を入力すると、最もよいパラメータとBICが出力される
def selectparameter(data,s):
    p=d=q=range(0,2)
    pdq=list(itertools.product(p,d,q))
    seasonal_pdq=[(x[0],x[1],x[2],s) for x in pdq]
    parameters=[]
    BICs=np.array([])
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod=sm.tsa.statespace.SARIMAX(convini_sales,order=param,seasonal_order=param_seasonal)
                results=mod.fit()
                parameters.append([param,param_seasonal,results.bic])
                BICs=np.append(BICs,results.bic)

            except:
                continue
                
    return parameters[np.argmin(BICs)]

#selectparameter(convini_sales,12)

In [ ]:
SARIMA_convini_sales=sm.tsa.statespace.SARIMAX(convini_sales,
                                       order=selectparameter(convini_sales,12)[0],
                                       seasonal_order=selectparameter(convini_sales,12)[1]).fit()

pred_1=SARIMA_convini_sales.predict("2015-01-31","2021-12-31")
pred_2=SARIMA_convini_sales.predict("2015-01-31","2050-12-31")

plt.figure(figsize=(8,8))

plt.subplot(2,1,1)
plt.plot(convini_sales)
plt.plot(pred_1,color="r")
plt.xlabel("date")
plt.ylabel("sales(million units)")

plt.subplot(2,1,2)
plt.plot(convini_sales)
plt.plot(pred_2,color="r")
plt.xlabel("date")
plt.ylabel("sales(million units)")

plt.show()